# Projet informatique - Optimisation d'un réseau de livraison

## Bou Hanna Emma, Clodion Sandra, Rossi Sophie

### Introduction

#### Définition du cadre d'étude
Un réseau de livraison est un système très complexe dans lequel interagissent de nombreux acteurs (clients, vendeurs, transporteurs …) et qui peut être plus ou moins étendu (échelle d’une commune, d’une ville, d’un pays …). Selon l’échelle prise en compte, les routes sont de taille différente et permettent ou pas le passage de camions: par exemple, dans les centres villes on peut imaginer une livraison par bicyclette. Ainsi, il y a différentes étapes dans le transport d’un colis qui passe par plusieurs entrepôts pour enfin être livré au client. 

   Dans le domaine de l'optimisation des réseaux de livraison, nous avons choisi de nous intéresser plus particulièrement au last mile delivery. Le last mile (en français, « dernier kilomètre ») correspond au dernier trajet de la marchandise vers son destinataire final (qu’il s’agisse d’un ménage, d’une entreprise ou d’un service public). Ce « dernier kilomètre » est la partie la plus coûteuse du transport de marchandises : on estime qu’il représente entre 20 et 30% du coût total de la livraison. Au-delà de l’aspect économique, la livraison du dernier kilomètre présente d’autres inconvénients majeurs, notamment un encombrement de la voirie dû à la circulation des véhicules transportant les marchandises et surtout à leur stationnement, mais aussi une pollution importante : les véhicules utilitaires légers utilisés pour les livraisons sont à l’origine de 30% des émissions de gaz à effet de serre en ville.

   Par ailleurs, le secteur du e-commerce a une importance de plus en plus grande au sein des enjeux liés à la livraison du dernier kilomètre et est en très forte expansion aujourd’hui. Nous voulions, au départ nous focaliser sur l’e-commerce, mais étant donnée l’absence de données, nous travaillerons avec des colis génériques (définis essentiellement par leur volume, et par le fait d’être non périssables), qui pourront tout à fait être transposés à l’e-commerce par la suite.

   Enfin, nous avons choisi de limiter notre étude à la région Île-de-France, territoire très urbain et concentrant une grande quantité d’entrepôts.

   Ainsi, nous choisissons de limiter notre sujet à l’optimisation du last mile delivery en Île-de-France. L’optimisation consistera essentiellement à minimiser le temps de trajet et les délais, tout en maximisant les stocks dans les camions.


### Réalisation du projet

#### Première Partie (cc Sandra je te laisse compléter <3)


#### Deuxième Partie (cc Sophie de même <3)

#### Troisième Partie : Mise en place d'un algorithme d'optimisation génétique

On a choisi d'implémenter un algorithme génétique pour optimiser le réseau de livraison modélisé par le graphe créé en partie 2.

##### Données nécessaires au bon fonctionnement de cet algorithme 
L'algorithme prend comme argument d'entrée les fichiers csv créés par la méthode generate_csv de la classe Graphe, ainsi que la liste renvoyée par cette méthode, qui contient le nom des entrepots ainsi que leur nombre maximum de camions et le nombre de colis à livrer dans la journée. 

De plus, il faut avoir accès à l'attribut garage.nb_camions de la classe Graphe pour avoir accès au nombre de camions disponibles!

Nous fixons également un coût de livraison de 10min.
On impose un coût par unité de distance égal à l'inverse de la vitesse moyenne en ville, soit 1/50.